In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Russian Ukrainian War.csv")

In [4]:
df = df[df["type"] == "Post"]

In [8]:
from langdetect import detect, LangDetectException

In [10]:
df["lang"] = df.apply(lambda x: detect_no_fail(x["text"]), axis=1)

/tmp/ipykernel_33461/1562491245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lang"] = df.apply(lambda x: detect_no_fail(x["text"]), axis=1)


In [9]:
def detect_no_fail(text):
    try:
        return detect(text)
    except LangDetectException:
        return None

In [12]:
df = df[df["lang"] == "en"]

In [14]:
df.shape

(8210, 22)

In [15]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')

/opt/conda/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [18]:
instructions = [
    ["Represent topic for clustering:", tweet]
    for tweet in df["text"].head(10)
]

In [25]:
embeddings = model.encode(instructions)
clustering_model = sklearn.cluster.MiniBatchKMeans(n_clusters=2)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)

[0 0 1 1 1 0 0 0 0 1]


/opt/conda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [24]:
import sklearn.cluster

In [28]:
clustering_model.get_feature_names_out()

array(['minibatchkmeans0', 'minibatchkmeans1'], dtype=object)

In [32]:
!pip install bertopic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for bertopic from https://files.pythonhosted.org/packages/06/49/f395e2e4d21dd49803494c8aec6087db61ea0ba211c6e5e57540b23334eb/bertopic-0.15.0-py2.py3-none-any.whl.metadata
  Using cached bertopic-0.15.0-py2.py3-none-any.whl.metadata (20 kB)
  Using cached hdbscan-0.8.33.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached umap_learn-0.5.4-py3-none-any.whl
  Obtaining dependency information for plotly>=4.7.0 from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
  Using cached plotly-5.18.0-py3-none-any.whl.metadata (7.0 kB)
  Obtaining dependency information for cython<3,>=0.27 from https://files.pythonhosted.org/packages/05/6a/bdc0773eeeab3e85b1adfd42cf95c325d024b919c9e1e0db75c82cc7bfc0/Cython-0.29.36-cp311-cp311-manylinux_2_17_aarch64.manylinux

In [50]:
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities=True, nr_topics="auto")

In [51]:
topics, probabilities = topic_model.fit_transform(df["text"])

In [52]:
len(set(topics))

81

In [53]:
len(topics)

8210

In [54]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [69]:
tokenizer = AutoTokenizer.from_pretrained("czearing/article-title-generator")
model = T5ForConditionalGeneration.from_pretrained("czearing/article-title-generator")

def generate_title(tweets):
    combined_tweets = ",\n\n\nTweet: ".join(tweets)
    inputs = tokenizer.encode("summarize: " + combined_tweets, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=3, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Group tweets by topics
grouped_tweets = {topic: [] for topic in set(topics)}
for tweet, topic in zip(df["text"].to_list(), topics):
    grouped_tweets[topic].append(tweet)

# Generate names for each topic
topic_names = {}
for topic, tweets in grouped_tweets.items():
    topic_name = generate_title(tweets)
    topic_names[topic] = topic_name

# Printing generated topic names
for topic, name in topic_names.items():
    print(f"Topic {topic}: {name}")

Topic 0: Ukrainian Army
Topic 1: Pope Francis
Topic 2: Russian losses
Topic 3: 
Topic 4: Ukraine'
Topic 5: Rhe
Topic 6: Israel and
Topic 7: Ukrainian Fe
Topic 8: Ukrainian President
Topic 9: 
Topic 10: The gall
Topic 11: These are
Topic 12: #Russia
Topic 13: Is
Topic 14: Ukrainian President
Topic 15: @H
Topic 16: '
Topic 17: War In
Topic 18: The Russian
Topic 19: Classical
Topic 20: The worst
Topic 21: Stocks
Topic 22: Latter
Topic 23: Ukrainian Legion
Topic 24: #DN
Topic 25: 
Topic 26: US lies
Topic 27: I dont
Topic 28: Ukraine-
Topic 29: Ukra
Topic 30: President of
Topic 31: RUS
Topic 32: 
Topic 33: Ukraine-
Topic 34: #ICC
Topic 35: Russia-
Topic 36: ISW
Topic 37: Kenyan
Topic 38: Over 1,000
Topic 39: Russia Ukraine
Topic 40: Russia’
Topic 41: Defender’
Topic 42: Sudan gum
Topic 43: Following Ukrainian
Topic 44: Russia'
Topic 45: Mariu
Topic 46: Ukrainian flag
Topic 47: Protest
Topic 48: Only 
Topic 49: Russian hackers
Topic 50: Anonymous declare
Topic 51: Hung
Topic 52: Ukraine war


In [65]:
topic_model.get_topic(3)

[('crimes', 0.06905809476001645),
 ('400', 0.05153306494585687),
 ('investigators', 0.050726075299617736),
 ('documented', 0.03990278264770596),
 ('than', 0.03273765726731776),
 ('president', 0.031032221138653362),
 ('zelenskyy', 0.029695405515336924),
 ('says', 0.028947455054580653),
 ('more', 0.027057110748453347),
 ('have', 0.026544717315707508)]

In [61]:
topic_number

0

In [71]:
grouped_tweets[0]

["Ukrainian Army's latest weapons against Russian forces in Ukraine | Russia-Ukraine war | WION\n#Russia #Ukraine #Britain #Americe",
 'Medvedev Says \'No Other #Option\' But Nuclear #War if #Ukrainian Counteroffensive Succeeds. Former #Russian president and current deputy Security Council chair warned of "global #nuclear fire" should Ukraine take back regions illegally annexed by #Moscow.',
 'Armed Forces of Ukraine had just arrived at their new positions then this happened \n#UkraineWarNews #RussianUkrainianWar #UkraineFrontLines',
 '#RussianUkrainianWar\n#RussiaIsATerroristState 🚀🚀🤬\n⛔ #StopRussia 🏴\u200d☠️🏴\u200d☠️ 🚀🚀🏴\u200d☠️🏴\u200d☠️❌\n#Kramatorsk 😰',
 'A counter offensive  on Russian territory \nwith UKRAINIAN MADE \nAerial and naval DRONES\nto  cross over ground battlefields completely MINED by Russia  is reaching its objectives\n in a new tactical warfare using  \nweapons MADE in Ukraine by its war industry  to hit back targets',
 '16 May: Russian Most Potent Weapon MEETS UKRA